In [1]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import json
from copy import copy

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Autosaving every 300 seconds


# Data Cleaning and Preliminary Exploration

## Preliminary Exploration

In [2]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)


(6937, 94)


In [3]:
# #### NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
# #### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

# # checking out the number of records for patients in ADNI2 phase, compared with other phases
# ADMCPATIENTDRUGCLASSES_20170512 = pd.read_csv('ADMCPATIENTDRUGCLASSES_20170512.csv')
# print(len(ADMCPATIENTDRUGCLASSES_20170512['Phase']))
# print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']))
# print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI3']))
# print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNIGO']))

# ADMCPATIENTDRUGCLASSES_20170512.head()

# drug_adni2 = ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']
# drug_adni2.shape

# drug_adni2.columns

# drug_adni2 = drug_adni2.rename(columns={'VISCODE2': 'VISCODE'})
# drug_adni2.columns
# pd.merge(ADNI2,drug_adni2, on=['RID', 'VISCODE'])

In [4]:
''' ADNI2 data within ADNIMERGE.csv '''
# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)



(6937, 94)


In [5]:
# It seems there were some 'unknown'..I replaced them with nan
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)
ADNI2 = ADNI2.replace(-1, np.nan)
ADNI2 = ADNI2.replace(-4, np.nan)

# From now on just bl


In [6]:
ADNI2_bl = ADNI2.copy()
ADNI2_bl = ADNI2_bl[ADNI2_bl['VISCODE'] == 'bl']

In [7]:
ADNI2_bl.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'FDG', 'PIB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
       'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl

In [8]:
ADNI2_bl.shape

(789, 94)

In [9]:
#Drop the ones w/o bl
#I kept 'RID', 'PTID', 'VISCODE', 'COLPROT','DX_bl'
#'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
#'PTMARRY', 'APOE4', 'FDG', , ADAS11_bl',
# 'ADAS13_bl', 'CDRSB_bl', 'MMSE_bl',
#'RAVLT_immediate_bl', 'RAVLT_learning_bl',
#'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
#'FLDSTRENG_bl', , 'Ventricles_bl', 'Hippocampus_bl',
#'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
#'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
#'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
#'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
#'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
#'PIB_bl', 'AV45_bl'


ADNI2_bl= ADNI2_bl.drop(labels = [ 'SITE', 'ORIGPROT', 'EXAMDATE',
         'AV45', 'CDRSB', 'ADAS11', 'ADAS13','PIB',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
       'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl',   'Years_bl', 'Month_bl', 'Month', 'M',
       'update_stamp', 'FSVERSION_bl', 'FLDSTRENG_bl'],axis=1)

# Drop beacuse same values 'FSVERSION_bl', 'FLDSTRENG_bl'

In [10]:
# calculate the % of each column that is NaNs and print in descending order
print((ADNI2_bl.isnull().sum()/len(ADNI2_bl['DX_bl'])).sort_values(ascending=False))

PIB_bl                      1.000000
Entorhinal_bl               0.136882
Fusiform_bl                 0.136882
MidTemp_bl                  0.136882
Hippocampus_bl              0.112801
Ventricles_bl               0.081115
EcogSPOrgan_bl              0.050697
WholeBrain_bl               0.046895
EcogSPDivatt_bl             0.032953
EcogPtOrgan_bl              0.030418
AV45_bl                     0.029151
EcogSPVisspat_bl            0.026616
EcogSPPlan_bl               0.022814
FDG                         0.019011
FDG_bl                      0.019011
ICV_bl                      0.017744
RAVLT_forgetting_bl         0.016477
RAVLT_learning_bl           0.013942
APOE4                       0.013942
EcogPtDivatt_bl             0.013942
EcogSPTotal_bl              0.013942
EcogSPMem_bl                0.012674
MOCA_bl                     0.012674
EcogSPLang_bl               0.011407
EcogPtVisspat_bl            0.010139
FAQ_bl                      0.007605
ADAS13_bl                   0.007605
E

In [11]:
#Drop 50% or more missing
ADNI2_bl= ADNI2_bl.drop(['PIB_bl'], axis=1)

In [12]:
encode = preprocessing.LabelEncoder()
ADNI2_bl.loc[:, 'DX_bl'] = encode.fit_transform(ADNI2_bl['DX_bl'])

In [13]:
ADNI2_bl.head()
print(ADNI2_bl.shape)
ADNI2_bl.columns

(789, 46)


Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTGENDER',
       'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl'],
      dtype='object')

In [14]:
#hot-one encoding for categoricla and clean

categorical_predictors = ['PTETHCAT', 'PTRACCAT', 'PTMARRY','APOE4', 'PTGENDER']



''' Remove all categorical data with nan (only ~0.5%) '''
missing_rows = np.nonzero(np.sum(ADNI2_bl[categorical_predictors].isnull(), axis=1))
ADNI2_bl = ADNI2_bl.drop(ADNI2_bl.index[missing_rows[0]])

'''' Hot-One Encoding of Categorical Predictors '''
ADNI2_bl_categorical = pd.get_dummies(ADNI2_bl[categorical_predictors] , columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
ADNI2_bl= pd.get_dummies(ADNI2_bl, columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
print(ADNI2_bl.shape)


(771, 58)


In [15]:
print(ADNI2_bl.columns)

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0',

### Removing All Missing Data

In [16]:
# ADNI2_bl_remove_all_missing = ADNI2_bl.copy()
# for column in ADNI2_bl.columns:
#     if ADNI2_bl[column].isnull().any():
#         ADNI2_remove_all_missing = ADNI2_bl.drop(labels=column, axis=1)

In [17]:
ADNI2_bl_remove = ADNI2_bl.copy()
print('Shape before delteing missing', ADNI2_bl_remove.shape)
ADNI2_bl_remove.dropna(axis=0, inplace=True)
print('Shape before delteing missing', ADNI2_bl_remove.shape) 
print('Data contains NaN?', ADNI2_bl_remove.isnull().any().any())

Shape before delteing missing (771, 58)
Shape before delteing missing (497, 58)
Data contains NaN? False


In [18]:
ADNI2_bl_remove.isnull().sum()

RID                           0
PTID                          0
VISCODE                       0
COLPROT                       0
DX_bl                         0
AGE                           0
PTEDUCAT                      0
FDG                           0
CDRSB_bl                      0
ADAS11_bl                     0
ADAS13_bl                     0
MMSE_bl                       0
RAVLT_immediate_bl            0
RAVLT_learning_bl             0
RAVLT_forgetting_bl           0
RAVLT_perc_forgetting_bl      0
FAQ_bl                        0
Ventricles_bl                 0
Hippocampus_bl                0
WholeBrain_bl                 0
Entorhinal_bl                 0
Fusiform_bl                   0
MidTemp_bl                    0
ICV_bl                        0
MOCA_bl                       0
EcogPtMem_bl                  0
EcogPtLang_bl                 0
EcogPtVisspat_bl              0
EcogPtPlan_bl                 0
EcogPtOrgan_bl                0
EcogPtDivatt_bl               0
EcogPtTo

In [19]:
ADNI2_bl_remove.head()

,RID,PTID,VISCODE,COLPROT,DX_bl,AGE,PTEDUCAT,FDG,CDRSB_bl,ADAS11_bl,...,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,PTGENDER_Female,PTGENDER_Male
91,5280,100_S_5280,bl,ADNI2,4,67.5,16,1.25195,0.0,3.0,...,0,0,0,1,0,1,0,0,0,1
92,5279,082_S_5279,bl,ADNI2,4,68.5,20,1.50629,0.0,4.0,...,1,0,1,0,0,1,0,0,0,1
142,5278,082_S_5278,bl,ADNI2,4,80.2,19,1.22045,0.0,3.0,...,1,0,0,1,0,0,1,0,0,1
174,5277,027_S_5277,bl,ADNI2,4,71.8,15,1.42276,0.5,4.0,...,1,0,0,0,1,0,1,0,1,0
360,5272,053_S_5272,bl,ADNI2,4,69.9,8,1.38841,0.0,5.0,...,1,0,1,0,0,1,0,0,1,0


In [20]:
ADNI2_bl_remove.shape

(497, 58)

In [21]:
ADNI2_bl_remove.columns

Index(['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0',

In [22]:
ADNI2_bl_remove.to_csv(path_or_buf='ADNI2_bl_remove_all_missing.csv', index=False)

## Model-Based Imputation 
- Drop around 5% of Categorical predictors and use Linear Regression for missing continuous predictors

In [23]:
def split (data, percentage):
    np.random.seed(9001) 
    msk = np.random.rand(len(data)) < percentage
    train = data[msk]
    test = data[~msk]
    return train, test

In [24]:
ADNI2_bl_impute = ADNI2_bl.copy()
ADNI2_bl_impute = ADNI2_bl_impute.drop(['VISCODE'], axis=1)

In [25]:
ADNI2_bl_impute.columns

Index(['RID', 'PTID', 'COLPROT', 'DX_bl', 'AGE', 'PTEDUCAT', 'FDG', 'CDRSB_bl',
       'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0

In [26]:
all_predictors = ['RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 'AGE', 'PTGENDER',
       'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0']

#For the continuos I had to remove
#'RID', 'PTID', 'VISCODE', 'COLPROT', 'DX_bl', 
#'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
#'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
#'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
#'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
#'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
#'APOE4_2.0'
    
continuous_predictors = ['AGE', 
       'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       ]

In [27]:
''' Create ADNI2_missing for imputation '''
ADNI2_bl_impute_remove_categorical = ADNI2_bl_impute[continuous_predictors]

''' Create the Dataframe for imputation '''
ADNI2_bl_missing = pd.concat([ADNI2_bl_impute_remove_categorical, ADNI2_bl_categorical], axis=1)


In [28]:
print(ADNI2_bl_impute_remove_categorical.shape)
print(ADNI2_bl_categorical.shape)

(771, 35)
(771, 17)


In [29]:
ADNI2_bl_missing['DX_bl']=ADNI2_bl_impute['DX_bl']
ADNI2_bl_missing.head()

,AGE,PTEDUCAT,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,...,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,PTGENDER_Female,PTGENDER_Male,DX_bl
58,66.9,17,0.0,10.0,15.0,29,42.0,5.0,2.0,20.0000,...,0,1,0,0,0,1,0,0,1,4
91,67.5,16,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0000,...,0,0,1,0,1,0,0,0,1,4
92,68.5,20,0.0,4.0,7.0,30,61.0,6.0,3.0,20.0000,...,0,1,0,0,1,0,0,0,1,4
142,80.2,19,0.0,3.0,10.0,29,36.0,8.0,7.0,63.6364,...,0,0,1,0,0,1,0,0,1,4
174,71.8,15,0.5,4.0,6.0,28,44.0,8.0,4.0,40.0000,...,0,0,0,1,0,1,0,1,0,4


In [30]:
print(ADNI2_bl_impute_remove_categorical.shape)

(771, 35)


In [31]:
print(ADNI2_bl_missing.columns)

Index(['AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
       'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
       'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
       'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
       'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
       'AV45_bl', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0', 'PTGENDER_Female', 'PTGEND

In [32]:
full_dataframe =  copy(ADNI2_bl_missing)
full_dataframe_train, full_dataframe_test = split(full_dataframe,0.75)
ADNI2_bl_missing_train, ADNI2_bl_missing_test = split(ADNI2_bl_missing,0.75)

In [33]:
#For train
missing_rows = []
for i in range(ADNI2_bl_missing_train.shape[0]):
    if pd.isnull(ADNI2_bl_missing_train.values[i]).any():
        missing_rows.append(i)
full_dataframe_train = full_dataframe_train.drop(full_dataframe_train.index[missing_rows], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_bl_missing_train)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe_train.shape))

''' To check that full_dataframe has no missing values '''
print(np.sum(np.sum(full_dataframe_train.isnull())))

The origin number of data: 575
The number of data that contains missing entries: 206
The dimension of full dataframe: (369, 53)
0


In [34]:
''' Model-Based Imputation - Linear Regression for train'''

lr = LinearRegression(fit_intercept = True)


for i in range(ADNI2_bl_missing_train.shape[0]):
    if ADNI2_bl_missing_train.iloc[i, :].isnull().any():
        missing_columns = ADNI2_bl_missing_train.columns[ADNI2_bl_missing_train.iloc[i, :].isnull()]
        #print(i, missing_columns)
        y_imp_train = full_dataframe_train[missing_columns]
        X_imp_train = full_dataframe_train.drop(missing_columns, 1)
#         print(y_imp.shape)
#         print(X_imp.shape)
        lr.fit(X_imp_train,y_imp_train)
        X_missing_train = ADNI2_bl_missing_train.iloc[i, :].drop(missing_columns)
        y_hat_train = lr.predict(X_missing_train)
        ADNI2_bl_missing_train.loc[ADNI2_bl_missing_train.index[i], missing_columns] = y_hat_train[0] 
     

In [35]:
ADNI2_bl_missing_train.isnull().sum()

AGE                           0
PTEDUCAT                      0
CDRSB_bl                      0
ADAS11_bl                     0
ADAS13_bl                     0
MMSE_bl                       0
RAVLT_immediate_bl            0
RAVLT_learning_bl             0
RAVLT_forgetting_bl           0
RAVLT_perc_forgetting_bl      0
FAQ_bl                        0
Ventricles_bl                 0
Hippocampus_bl                0
WholeBrain_bl                 0
Entorhinal_bl                 0
Fusiform_bl                   0
MidTemp_bl                    0
ICV_bl                        0
MOCA_bl                       0
EcogPtMem_bl                  0
EcogPtLang_bl                 0
EcogPtVisspat_bl              0
EcogPtPlan_bl                 0
EcogPtOrgan_bl                0
EcogPtDivatt_bl               0
EcogPtTotal_bl                0
EcogSPMem_bl                  0
EcogSPLang_bl                 0
EcogSPVisspat_bl              0
EcogSPPlan_bl                 0
EcogSPOrgan_bl                0
EcogSPDi

In [36]:
#For test
missing_rows_test = []
for i in range(ADNI2_bl_missing_test.shape[0]):
    if pd.isnull(ADNI2_bl_missing_test.values[i]).any():
        missing_rows_test.append(i)
full_dataframe_test = full_dataframe_test.drop(full_dataframe_test.index[missing_rows_test], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_bl_missing_test)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe_test.shape))

''' To check that full_dataframe has no missing values '''
print(np.sum(np.sum(full_dataframe_test.isnull())))

The origin number of data: 196
The number of data that contains missing entries: 206
The dimension of full dataframe: (128, 53)
0


In [37]:
#Model-Based Imputation - Linear Regression for test
#Iused the same model as in the train

lr = LinearRegression(fit_intercept = True)

for i in range(ADNI2_bl_missing_test.shape[0]):
    if ADNI2_bl_missing_test.iloc[i, :].isnull().any():
        missing_columns = ADNI2_bl_missing_test.columns[ADNI2_bl_missing_test.iloc[i, :].isnull()]
        y_imp_test = full_dataframe_test[missing_columns]
        X_imp_test = full_dataframe_test.drop(missing_columns, 1)
        #print(i, missing_columns)
#         print(y_imp.shape)
#         print(X_imp.shape)
        lr.fit(X_imp_test,y_imp_test)
        X_missing_test = ADNI2_bl_missing_test.iloc[i, :].drop(missing_columns)
        y_hat_test = lr.predict(X_missing_test)
        ADNI2_bl_missing_test.loc[ADNI2_bl_missing_test.index[i], missing_columns] = y_hat_test[0] 
     

In [38]:
print(ADNI2_bl_missing_train.shape)
print(ADNI2_bl_missing_test.shape)
print(ADNI2_bl_missing_test.shape)
print(ADNI2_bl_remove.shape)

(575, 53)
(196, 53)
(196, 53)
(497, 58)


In [39]:
print(ADNI2_bl_missing_train.shape)
print(ADNI2_bl_missing_test.shape)

(575, 53)
(196, 53)


In [40]:
ADNI2_bl_missing_train.isnull().sum()

AGE                           0
PTEDUCAT                      0
CDRSB_bl                      0
ADAS11_bl                     0
ADAS13_bl                     0
MMSE_bl                       0
RAVLT_immediate_bl            0
RAVLT_learning_bl             0
RAVLT_forgetting_bl           0
RAVLT_perc_forgetting_bl      0
FAQ_bl                        0
Ventricles_bl                 0
Hippocampus_bl                0
WholeBrain_bl                 0
Entorhinal_bl                 0
Fusiform_bl                   0
MidTemp_bl                    0
ICV_bl                        0
MOCA_bl                       0
EcogPtMem_bl                  0
EcogPtLang_bl                 0
EcogPtVisspat_bl              0
EcogPtPlan_bl                 0
EcogPtOrgan_bl                0
EcogPtDivatt_bl               0
EcogPtTotal_bl                0
EcogSPMem_bl                  0
EcogSPLang_bl                 0
EcogSPVisspat_bl              0
EcogSPPlan_bl                 0
EcogSPOrgan_bl                0
EcogSPDi

In [41]:
ADNI2_bl_missing_test.isnull().sum()

AGE                           0
PTEDUCAT                      0
CDRSB_bl                      0
ADAS11_bl                     0
ADAS13_bl                     0
MMSE_bl                       0
RAVLT_immediate_bl            0
RAVLT_learning_bl             0
RAVLT_forgetting_bl           0
RAVLT_perc_forgetting_bl      0
FAQ_bl                        0
Ventricles_bl                 0
Hippocampus_bl                0
WholeBrain_bl                 0
Entorhinal_bl                 0
Fusiform_bl                   0
MidTemp_bl                    0
ICV_bl                        0
MOCA_bl                       0
EcogPtMem_bl                  0
EcogPtLang_bl                 0
EcogPtVisspat_bl              0
EcogPtPlan_bl                 0
EcogPtOrgan_bl                0
EcogPtDivatt_bl               0
EcogPtTotal_bl                0
EcogSPMem_bl                  0
EcogSPLang_bl                 0
EcogSPVisspat_bl              0
EcogSPPlan_bl                 0
EcogSPOrgan_bl                0
EcogSPDi

In [43]:
ADNI2_bl_missing_train.to_csv(path_or_buf='ADNI2_bl_impute_train.csv', index=False)
ADNI2_bl_missing_test.to_csv(path_or_buf='ADNI2_bl_impute_test.csv', index=False)

# Below are More Preliminary Exploration by Carlo

                                                 **STRATEGY**

'RID',                    They should not be used for prediction, I'd keep them just for merging the prediction?
'PTID',                   They should not be used for prediction, I'd keep them just for merging the prediction?      
'VISCODE'                 They should not be used for prediction, I'd keep them just for merging the prediction?        'SITE'                    Shall we discard for the prediction?
'COLPROT'                 Shall we discard for the prediction?
'ORIGPROT'                Shall we discard for the prediction?
'EXAMDATE'                Shall we discard for the prediction?
'DX_bl'                   Hotone, categorical
'AGE',                    OK, int
'PTGENDER'                Hotone, categorical
'PTEDUCAT'                OK, int
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'                 Hotone, categorical
'APOE4'                   Hotone, categorical   Class of protein
'CDRSB'                   OK,float              Clinical Dementia Rating Scale–Sum of Boxes
'ADAS11'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'ADAS13'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'MMSE',                   OK,float              Mini Mental State Examination
'RAVLT_immediate'         OK,float              Rey Auditory Verbal Learning Test
'RAVLT_learning'          OK,float              Rey Auditory Verbal Learning Test
'RAVLT_forgetting',       OK,float              Rey Auditory Verbal Learning Test
'RAVLT_perc_forgetting'   OK,float              Rey Auditory Verbal Learning Test
'FAQ'                     OK,float              Functional Assessment Questionnaire
'MOCA'                    OK,float              Montreal Cognitive Assessment (MoCA)
'EcogPtMem'               OK,float              Cognitive
'EcogPtLang',             OK,float              Cognitive
'EcogPtVisspat'           OK,float              Cognitive
'EcogPtPlan'              OK,float              Cognitive
'EcogPtOrgan'             OK,float              Cognitive
'EcogPtDivatt'            OK,float              Cognitive
'EcogPtTotal'             OK,float              Cognitive
'EcogSPMem'               OK,float              Cognitive
'EcogSPLang'              OK,float              Cognitive
'EcogSPVisspat'           OK,float              Cognitive
'EcogSPPlan',             OK,float              Cognitive
'EcogSPOrgan'             OK,float              Cognitive
'EcogSPDivatt'            OK,float              Cognitive
'EcogSPTotal'             OK,float              Cognitive
'FSVERSION',              Just one value, discard
'Hippocampus'             OK,float              Hippocampus volume
'WholeBrain'              OK,float              Brain volume
'ICV'                     OK,float              Intracranial volume
'DX'                      OUR Y, ENCODING
'EXAMDATE_bl'             Shall we discard for the prediction?
'CDRSB_bl'                       ***FROM HERE BASELINE***
'ADAS11_bl'
'ADAS13_bl'
'MMSE_bl'
'RAVLT_immediate_bl'
'RAVLT_learning_bl'
'RAVLT_forgetting_bl'
'RAVLT_perc_forgetting_bl',
'FAQ_bl', 'FLDSTRENG_bl'
'FSVERSION_bl'
'Ventricles_bl',
'Hippocampus_bl'
'WholeBrain_bl'
'Entorhinal_bl'
'Fusiform_bl',
'MidTemp_bl'
'ICV_bl'
'MOCA_bl'
'EcogPtMem_bl'
'EcogPtLang_bl',
'EcogPtVisspat_bl'
'EcogPtPlan_bl'
'EcogPtOrgan_bl',
'EcogPtDivatt_bl'
'EcogPtTotal_bl'
'EcogSPMem_bl'
'EcogSPLang_bl',
'EcogSPVisspat_bl'
'EcogSPPlan_bl'
'EcogSPOrgan_bl',
'EcogSPDivatt_bl'
'EcogSPTotal_bl'
'FDG_bl'                   OK,float               fluorescence connect to PET
'AV45_bl'                  OK,float               average survival
'Years_bl',                OK,float               years form baseline
'Month_bl'                 Shall we discard for the prediction? We already have 'Years_bl'
'Month'                    Shall we discard for the prediction? We already have 'Years_bl'
'M'                        Shall we discard for the prediction? We already have 'Years_bl'
'update_stamp              Shall we discard for the prediction? 